In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import pandas as pd
root = p.Path("../data/algorithms_out/clustering/data_Clustering_Aloise benchmark.json").resolve()
root_folder = p.Path("../").resolve()
df = pd.read_json(root)

In [3]:
list_cases_per_init = {}
for initialization, df_init in df.groupby("IMPR"):
    L_cases = []
    for instance, df_instance in df_init.groupby("INSTANCE"):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        L_cases.append(case)

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases


problem = "mssc"
dataset = "benchmark"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)